# Code The Backpropagation


Esta implementação foi baseada no exemplo do notebook backpropagation passo-a-passo, tanto os dados quanto as ideias.

In [24]:
import numpy as np

In [25]:
class Neuron:
    def __init__(self, weights, bias):
        self.weights = weights
        self.bias = bias
        self.total_weight_errors = []
        
    def calculate_output(self, inputs):
        self.inputs = inputs
        self.output = self.activation(self.net(inputs))
        return self.output
        
    # Realiza o calculo de soma(W*x + b)    
    def net(self, inputs):
        return np.dot(self.weights, inputs) + self.bias    
    
    # Função de ativação, no caso estamos utilizando a função logistica f(x) = 1/(1 + exp(-x)) que gera curva sigmoide
    def activation(self, net):
        return 1.0 / (1.0 + np.exp(-net))

    # Derivada da função logistica, se f'(x) = f(x)(1 - f(x))
    def derivative_activation(self):
        return self.output*(1 - self.output)
    
    def calculate_delta(self, target):
        self.delta = (self.output - target)*self.derivative_activation() 
        return self.delta
    
    def update_weights(self, learning_rate):
        for i in range(0, len(self.weights)):
            self.weights[i] -= learning_rate * self.total_weight_errors[i]
        

In [26]:
class NeuralNetwork:
    LEARNING_RATE = 0.5
    
    # Número de entradas de dados, número de nós na camada oculta e número de nós de saida.
    def __init__(self, num_inputs, num_hidden, num_outputs):
        self.num_inputs = num_inputs
        self.num_hidden = num_hidden
        self.num_outputs = num_outputs
        
        # OBS: Dados de teste do artigo que serviu de base
#         self.hidden_layer = [ Neuron([.15, .20], .35), Neuron([.25, .30], .35) ]
#         self.output_layer = [ Neuron([.40, .45], .60), Neuron([.50, .55], .60) ]
        
        # As camadas são um conjunto de neuronios
        self.hidden_layer = self.init_layer_weights(num_inputs, num_hidden)
        self.output_layer = self.init_layer_weights(num_hidden, num_outputs)
        
    # Inicia as camadas de neuronios com pesos gerados aleatoriamente para cada neuronio     
    def init_layer_weights(self, num_weights, num_neurons):
        neurons_layer = []
        for i in range(0, num_neurons):
            weights = np.random.randn(num_weights) * 0.05 
            bias = (np.random.randn(1) * 0.05)[0] 
            neurons_layer.append(Neuron(weights, bias))
        return neurons_layer
    
    
    def backpropagation(self, inputs_list, targets):
        for inputs in inputs_list, target:        
            prediction = self.forward_pass(inputs)
            print("Prediction: ", prediction)

            error = self.calculate_total_error(targets, prediction)
            print("\nError: ", error)

            print("\n----- Pesos antes -----")
            self.show_weights()

            self.backward_pass(target)

            print("\n----- Pesos depois ----- ")
            self.show_weights()
        
    def forward_pass(self, inputs):
        outputs_hidden = self.feed_forward(inputs, self.hidden_layer)
        return self.feed_forward(outputs_hidden, self.output_layer)
    
    def feed_forward(self, inputs, neurons):
        outputs = []
        for neuron in neurons:
            outputs.append(neuron.calculate_output(inputs))
        return outputs
    
    def backward_pass(self, target):
        # Calcula os deltas para todos pesos (weights) da hidden layer para output layer
        self.compute_output_deltas(target)
        # Calcula os deltas para todos pesos (weights) da input layer para hidden layer
        self.compute_hidden_deltas()
        
        # Atualização dos pesos, só pode ser executado se o calculo dos deltas for realizado antes
        self.update_weights(self.output_layer)
        self.update_weights(self.hidden_layer)
        
    # calculando deltas para a camada de saida (output layer)    
    def compute_output_deltas(self, targets):
        for neuron, target in zip(self.output_layer, targets):
            neuron.calculate_delta(target)
        
        for o_neuron in self.output_layer:
            for h_neuron, o_weights in zip(self.hidden_layer, o_neuron.weights):
                total_weight_error = o_neuron.delta * h_neuron.output
                o_neuron.total_weight_errors.append(total_weight_error)
        
    def update_weights(self, layer):
        for neuron in layer:
            neuron.update_weights(self.LEARNING_RATE)
        
    def compute_total_deltas_weights(self, index):
        total_delta_weight = 0
        for neuron in nn.output_layer:
            total_delta_weight += neuron.delta * neuron.weights[index]
        return total_delta_weight
    
    def compute_hidden_deltas(self):
        for i, neuron in enumerate(self.hidden_layer):
            for j in range(0, len(neuron.weights)):
                total_deltas_weights = self.compute_total_deltas_weights(i)            
                total_weight_error = total_deltas_weights * neuron.derivative_activation() * neuron.inputs[j]
                neuron.total_weight_errors.append(total_weight_error)
    
    def calculate_error(self, target, output):
        return (0.5)*(target - output)**2
    
    def calculate_total_error(self, targets, outputs):
        total_error = 0.0
        for target, output in zip(targets, outputs):
            total_error += self.calculate_error(target, output)
        return total_error
    
    def show_weights(self):
        print("\nOutput Layer: ")
        for neuron in self.output_layer:
            print(neuron.weights)
        
        print("\nHidden Layer: ")
        for neuron in self.hidden_layer:
            print(neuron.weights)

Para testar, utilizaremos os seguintes dados do exemplo.

In [27]:
# inputs_list = [[.05, .10]]
# targets = [.01, .99]

# nn = NeuralNetwork(num_inputs=2, num_hidden=2, num_outputs=2)
# nn.backpropagation(inputs_list, targets)

Atualização de pesos para Output Layer:
[0.358916479, 0.408666186]
[0.511301270, 0.561370121]

Atualização de pesos para Hidden Layer:
[0.149780716, 0.19956143]
[0,24975114, 0,29950229]

In [28]:

from sklearn import datasets
iris = datasets.load_iris()

X = iris.data
y = iris.target

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .5)

nn = NeuralNetwork(num_inputs=4, num_hidden=2, num_outputs=3)
for i in range(0, 10):
    nn.backpropagation(X_train, y_train)

ValueError: shapes (4,) and (75,4) not aligned: 4 (dim 0) != 75 (dim 0)

In [ ]:
nn.forward_pass(X_test[0])